## song_meta & crawling 데이터프레임

<br>

1) 데이터 불러오기

In [1]:
from tqdm import tqdm_notebook
from selenium import webdriver
import requests
import urllib

import numpy as np
import pandas as pd

#################### song_meta_filter.csv 있는 자기경로 ####################

In [2]:
song_meta = pd.read_csv('C:/Users/user/Desktop/final/song_meta_filter.csv', encoding='utf-8', index_col=0)

In [3]:
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,"['GN1102', 'GN1101']",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,['GN1100'],['Matoma'],3
1,['GN1201'],20170407,Luv.Loops,10053652,[1625859],Hi (Heyoo),['GN1200'],['Miraa.'],10
2,['GN1301'],20080610,Classic Masters,133952,[104493],Everything We Do (2002 Digital Remaster),['GN1300'],['Rene & Angela'],11
3,"['GN2704', 'GN1104', 'GN1103', 'GN1101']",20140728,Knock You Out (Remixes),2269860,[455705],Knock You Out,"['GN2700', 'GN1100']",['Bingo Players'],14
4,"['GN0805', 'GN0501', 'GN0502', 'GN0801', 'GN05...",20150205,내가 부른 그림 2,2303168,[230399],무얼 기다리나 (Feat. 조원선),"['GN0500', 'GN0800']",['이영훈'],19
...,...,...,...,...,...,...,...,...,...
92981,"['GN0805', 'GN0801']",20120604,그저 그런 오후,2125663,[593714],그저 그런 오후,['GN0800'],['소요'],707927
92982,['GN0601'],20141224,넥스트 신해철 Reboot Yourself,2296666,[845],Hope,['GN0600'],['신해철'],707932
92983,"['GN0702', 'GN0708', 'GN0701']",20200313,내일은 미스터트롯 결승전 베스트,10402513,[2763943],나는 남자다,['GN0700'],['김희재'],707949
92984,"['GN2704', 'GN1104', 'GN1103', 'GN1101']",20131008,Revolution,2210541,[487526],Biggie Bounce,"['GN2700', 'GN1100']",['Diplo'],707964


In [4]:
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92986 entries, 0 to 92985
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   song_gn_dtl_gnr_basket  92986 non-null  object
 1   issue_date              92986 non-null  int64 
 2   album_name              92984 non-null  object
 3   album_id                92986 non-null  int64 
 4   artist_id_basket        92986 non-null  object
 5   song_name               92985 non-null  object
 6   song_gn_gnr_basket      92986 non-null  object
 7   artist_name_basket      92986 non-null  object
 8   id                      92986 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 7.1+ MB


총 707,989곡의 메타 정보가 담겨 있으며 포함하는 값은 아래와 같습니다.
- song_gn_dtl_gnr_basket : 상세 장르 코드
- issue_date : 곡 발매 일자 (yyyymmdd)
- album_name : 앨범 명
- album_id : 앨범 아이디
- artist_id_basket : 아티스트 아이디 (복수일 경우 띄어쓰기로 구분)
- song_name : 곡 명
- song_gn_gnr_basket : 대분류 장르코드
- artist_name_basket : 아티스트 명 (복수일 경우 띄어쓰기로 구분)
- id : 곡 아이디

<br>

2) 크롤링을 위한 데이터 전처리

In [5]:
song_meta['artist_name_basket'] = song_meta['artist_name_basket'].map(lambda x: str(x).lstrip('[').rstrip(']'))
song_meta['search'] = song_meta['artist_name_basket'].map(str) + " " + song_meta['song_name']

<br>

# 3) 개인 데이터셋 범위 지정

#################### 크롤링할 범위 설정 ####################

In [6]:
start = 0
end = 10

data = song_meta[start:end]
data

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,search
0,"['GN1102', 'GN1101']",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,['GN1100'],'Matoma',3,'Matoma' Feeling Right (Everything Is Nice) (F...
1,['GN1201'],20170407,Luv.Loops,10053652,[1625859],Hi (Heyoo),['GN1200'],'Miraa.',10,'Miraa.' Hi (Heyoo)
2,['GN1301'],20080610,Classic Masters,133952,[104493],Everything We Do (2002 Digital Remaster),['GN1300'],'Rene & Angela',11,'Rene & Angela' Everything We Do (2002 Digital...
3,"['GN2704', 'GN1104', 'GN1103', 'GN1101']",20140728,Knock You Out (Remixes),2269860,[455705],Knock You Out,"['GN2700', 'GN1100']",'Bingo Players',14,'Bingo Players' Knock You Out
4,"['GN0805', 'GN0501', 'GN0502', 'GN0801', 'GN05...",20150205,내가 부른 그림 2,2303168,[230399],무얼 기다리나 (Feat. 조원선),"['GN0500', 'GN0800']",'이영훈',19,'이영훈' 무얼 기다리나 (Feat. 조원선)
5,"['GN1501', 'GN1504']",20170617,비밀의 숲 OST Part.2,10072040,[1759976],먼지,['GN1500'],'에버루아',32,'에버루아' 먼지
6,"['GN1003', 'GN1001']",20170217,Bed Of Liars,10039414,[726278],Your Love Could Start A War,['GN1000'],'The Unlikely Candidates',40,'The Unlikely Candidates' Your Love Could Star...
7,"['GN0509', 'GN0501', 'GN0304', 'GN0505', 'GN03...",20140828,Clarity,2278112,[588331],Walk Alone,"['GN0500', 'GN0300']",'LHA',47,'LHA' Walk Alone
8,"['GN0801', 'GN0803']",20051212,김광석 Best,311488,[2960],나의 노래,['GN0800'],'김광석',48,'김광석' 나의 노래
9,['GN0101'],20190408,있어줄래,10271322,[500421],있어줄래,['GN0100'],'길구봉구',54,'길구봉구' 있어줄래


<br>

4) 크롤링

#################### chromedriver 있는 자기경로 ####################

In [7]:
driver = webdriver.Chrome("C:/chromedriver.exe")

### 유튜브 크롤링

- 가수 + 제목 검색 시, 해당 Title과 Url 추출

In [8]:
def get_youtube_with_keyword(keyword) :
    
    search_keyword_encode = requests.utils.quote(keyword)
    url = "https://www.youtube.com/results?search_query=" + search_keyword_encode

    driver.get(url)

    elements = driver.find_elements_by_css_selector('h3.title-and-badge.style-scope.ytd-video-renderer')
    href = ''
#     title = ''

    for idx, element in enumerate(elements) :
        if idx == 0 :
            href = element.find_elements_by_tag_name('a')[0].get_attribute('href')
#             title = element.find_elements_by_tag_name('a')[0].get_attribute('title')

    return href

In [9]:
href = []

for i in tqdm_notebook(range(start,end)) :
    
    result = get_youtube_with_keyword(data['search'][i])
    href.append(result)

<ipython-input-9-7344f064ffc2>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(start,end)) :


<br>

In [10]:
data['url'] = href
data.drop(['search'], axis='columns', inplace=True)

<ipython-input-10-101fae91ccad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['url'] = href


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,search,url
0,"['GN1102', 'GN1101']",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,['GN1100'],'Matoma',3,'Matoma' Feeling Right (Everything Is Nice) (F...,https://www.youtube.com/watch?v=QRxdrus-I5c
1,['GN1201'],20170407,Luv.Loops,10053652,[1625859],Hi (Heyoo),['GN1200'],'Miraa.',10,'Miraa.' Hi (Heyoo),https://www.youtube.com/watch?v=FzdZ06lw_HU
2,['GN1301'],20080610,Classic Masters,133952,[104493],Everything We Do (2002 Digital Remaster),['GN1300'],'Rene & Angela',11,'Rene & Angela' Everything We Do (2002 Digital...,https://www.youtube.com/watch?v=HpvWOh3QBCE
3,"['GN2704', 'GN1104', 'GN1103', 'GN1101']",20140728,Knock You Out (Remixes),2269860,[455705],Knock You Out,"['GN2700', 'GN1100']",'Bingo Players',14,'Bingo Players' Knock You Out,https://www.youtube.com/watch?v=-LR1ONlQC7o
4,"['GN0805', 'GN0501', 'GN0502', 'GN0801', 'GN05...",20150205,내가 부른 그림 2,2303168,[230399],무얼 기다리나 (Feat. 조원선),"['GN0500', 'GN0800']",'이영훈',19,'이영훈' 무얼 기다리나 (Feat. 조원선),https://www.youtube.com/watch?v=mqgKywvzSRk
5,"['GN1501', 'GN1504']",20170617,비밀의 숲 OST Part.2,10072040,[1759976],먼지,['GN1500'],'에버루아',32,'에버루아' 먼지,https://www.youtube.com/watch?v=lYzseY9kuPA
6,"['GN1003', 'GN1001']",20170217,Bed Of Liars,10039414,[726278],Your Love Could Start A War,['GN1000'],'The Unlikely Candidates',40,'The Unlikely Candidates' Your Love Could Star...,https://www.youtube.com/watch?v=UrmgBrIfLcM
7,"['GN0509', 'GN0501', 'GN0304', 'GN0505', 'GN03...",20140828,Clarity,2278112,[588331],Walk Alone,"['GN0500', 'GN0300']",'LHA',47,'LHA' Walk Alone,https://www.youtube.com/watch?v=wbeQVzXvtok
8,"['GN0801', 'GN0803']",20051212,김광석 Best,311488,[2960],나의 노래,['GN0800'],'김광석',48,'김광석' 나의 노래,https://www.youtube.com/watch?v=SXJ3QBMFkcg
9,['GN0101'],20190408,있어줄래,10271322,[500421],있어줄래,['GN0100'],'길구봉구',54,'길구봉구' 있어줄래,https://www.youtube.com/watch?v=3xuut5mZFcg


In [12]:
data

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,url
0,"['GN1102', 'GN1101']",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,['GN1100'],'Matoma',3,https://www.youtube.com/watch?v=QRxdrus-I5c
1,['GN1201'],20170407,Luv.Loops,10053652,[1625859],Hi (Heyoo),['GN1200'],'Miraa.',10,https://www.youtube.com/watch?v=FzdZ06lw_HU
2,['GN1301'],20080610,Classic Masters,133952,[104493],Everything We Do (2002 Digital Remaster),['GN1300'],'Rene & Angela',11,https://www.youtube.com/watch?v=HpvWOh3QBCE
3,"['GN2704', 'GN1104', 'GN1103', 'GN1101']",20140728,Knock You Out (Remixes),2269860,[455705],Knock You Out,"['GN2700', 'GN1100']",'Bingo Players',14,https://www.youtube.com/watch?v=-LR1ONlQC7o
4,"['GN0805', 'GN0501', 'GN0502', 'GN0801', 'GN05...",20150205,내가 부른 그림 2,2303168,[230399],무얼 기다리나 (Feat. 조원선),"['GN0500', 'GN0800']",'이영훈',19,https://www.youtube.com/watch?v=mqgKywvzSRk
5,"['GN1501', 'GN1504']",20170617,비밀의 숲 OST Part.2,10072040,[1759976],먼지,['GN1500'],'에버루아',32,https://www.youtube.com/watch?v=lYzseY9kuPA
6,"['GN1003', 'GN1001']",20170217,Bed Of Liars,10039414,[726278],Your Love Could Start A War,['GN1000'],'The Unlikely Candidates',40,https://www.youtube.com/watch?v=UrmgBrIfLcM
7,"['GN0509', 'GN0501', 'GN0304', 'GN0505', 'GN03...",20140828,Clarity,2278112,[588331],Walk Alone,"['GN0500', 'GN0300']",'LHA',47,https://www.youtube.com/watch?v=wbeQVzXvtok
8,"['GN0801', 'GN0803']",20051212,김광석 Best,311488,[2960],나의 노래,['GN0800'],'김광석',48,https://www.youtube.com/watch?v=SXJ3QBMFkcg
9,['GN0101'],20190408,있어줄래,10271322,[500421],있어줄래,['GN0100'],'길구봉구',54,https://www.youtube.com/watch?v=3xuut5mZFcg


# 개인 컴퓨터 경로 수정

#################### 크롤링한 데이터 저장할 자기경로 ####################
#################### 할때마다 csv 이름 변경해서 저장 ####################

In [13]:
data.to_csv('C:/Users/user/Desktop/final/song_youtube_meta1.csv', encoding='utf-8-sig')

In [14]:
data

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,url
0,"['GN1102', 'GN1101']",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,['GN1100'],'Matoma',3,https://www.youtube.com/watch?v=QRxdrus-I5c
1,['GN1201'],20170407,Luv.Loops,10053652,[1625859],Hi (Heyoo),['GN1200'],'Miraa.',10,https://www.youtube.com/watch?v=FzdZ06lw_HU
2,['GN1301'],20080610,Classic Masters,133952,[104493],Everything We Do (2002 Digital Remaster),['GN1300'],'Rene & Angela',11,https://www.youtube.com/watch?v=HpvWOh3QBCE
3,"['GN2704', 'GN1104', 'GN1103', 'GN1101']",20140728,Knock You Out (Remixes),2269860,[455705],Knock You Out,"['GN2700', 'GN1100']",'Bingo Players',14,https://www.youtube.com/watch?v=-LR1ONlQC7o
4,"['GN0805', 'GN0501', 'GN0502', 'GN0801', 'GN05...",20150205,내가 부른 그림 2,2303168,[230399],무얼 기다리나 (Feat. 조원선),"['GN0500', 'GN0800']",'이영훈',19,https://www.youtube.com/watch?v=mqgKywvzSRk
5,"['GN1501', 'GN1504']",20170617,비밀의 숲 OST Part.2,10072040,[1759976],먼지,['GN1500'],'에버루아',32,https://www.youtube.com/watch?v=lYzseY9kuPA
6,"['GN1003', 'GN1001']",20170217,Bed Of Liars,10039414,[726278],Your Love Could Start A War,['GN1000'],'The Unlikely Candidates',40,https://www.youtube.com/watch?v=UrmgBrIfLcM
7,"['GN0509', 'GN0501', 'GN0304', 'GN0505', 'GN03...",20140828,Clarity,2278112,[588331],Walk Alone,"['GN0500', 'GN0300']",'LHA',47,https://www.youtube.com/watch?v=wbeQVzXvtok
8,"['GN0801', 'GN0803']",20051212,김광석 Best,311488,[2960],나의 노래,['GN0800'],'김광석',48,https://www.youtube.com/watch?v=SXJ3QBMFkcg
9,['GN0101'],20190408,있어줄래,10271322,[500421],있어줄래,['GN0100'],'길구봉구',54,https://www.youtube.com/watch?v=3xuut5mZFcg
